# Geospatial Data Loading Example

This notebook demonstrates how to use the geospatial data loaders for the Cabruca Segmentation project.

In [ ]:
# Setup
import sys
from pathlib import Path

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

# Import simple loader (always available)
from src.data_processing import (
    SimpleGeoJSONLoader,
    CoordinateTransformer,
    ImageryMetadata,
    BandCalculations,
    FULL_LOADER_AVAILABLE
)

print(f"Full loader available: {FULL_LOADER_AVAILABLE}")

## 1. Loading GeoJSON Files

In [ ]:
# Load farm1.geojson
farm1_path = Path.cwd().parent / "farm1.geojson"
if farm1_path.exists():
    loader = SimpleGeoJSONLoader(farm1_path)
    farm1_aois = loader.extract_aois()
    
    print(f"Found {len(farm1_aois)} AOI(s) in farm1.geojson\n")
    
    for aoi in farm1_aois:
        print(f"Name: {aoi.name}")
        print(f"Center: {aoi.center[0]:.4f}, {aoi.center[1]:.4f}")
        print(f"Area: {aoi.area_approx_hectares:.2f} hectares")
        print(f"Bounds: {aoi.bounds}")
        print(f"Properties: {list(aoi.properties.keys())}")
else:
    print(f"File not found: {farm1_path}")

In [ ]:
# Load camacan-geo.geojson
camacan_path = Path.cwd().parent / "camacan-geo.geojson"
if camacan_path.exists():
    loader = SimpleGeoJSONLoader(camacan_path)
    camacan_aois = loader.extract_aois()
    
    print(f"Found {len(camacan_aois)} AOI(s) in camacan-geo.geojson\n")
    
    for aoi in camacan_aois:
        print(f"Name: {aoi.name}")
        print(f"Center: {aoi.center[0]:.4f}, {aoi.center[1]:.4f}")
        print(f"Area: {aoi.area_approx_hectares:.2f} hectares")
        print(f"Bounds: {aoi.bounds}")

## 2. Planet Labs AOI Format

In [ ]:
# Convert to Planet Labs format
if farm1_path.exists():
    loader = SimpleGeoJSONLoader(farm1_path)
    planet_aoi = loader.to_planet_format(0)
    
    print("Planet Labs AOI format:")
    print(f"Type: {planet_aoi['type']}")
    print(f"Number of vertices: {len(planet_aoi['coordinates'][0])}")
    print(f"First coordinate: {planet_aoi['coordinates'][0][0]}")
    
    # This format can be used directly with Planet Labs API
    import json
    print("\nJSON representation:")
    print(json.dumps(planet_aoi, indent=2))

## 3. Coordinate Conversions

In [ ]:
# Demonstrate coordinate conversion
if farm1_aois:
    aoi = farm1_aois[0]
    
    # Simulate an image covering the AOI
    image_width = 1000
    image_height = 1000
    
    # Convert AOI center to pixel coordinates
    center_lon, center_lat = aoi.center
    
    pixel_x, pixel_y = CoordinateTransformer.geo_to_pixel_simple(
        center_lon, center_lat,
        aoi.bounds,
        image_width, image_height
    )
    
    print(f"AOI Center (Geographic): {center_lon:.6f}, {center_lat:.6f}")
    print(f"AOI Center (Pixel): {pixel_x}, {pixel_y}")
    
    # Convert back to geographic
    back_lon, back_lat = CoordinateTransformer.pixel_to_geo_simple(
        pixel_x, pixel_y,
        aoi.bounds,
        image_width, image_height
    )
    
    print(f"\nBack to Geographic: {back_lon:.6f}, {back_lat:.6f}")
    print(f"Conversion error: {abs(back_lon - center_lon):.8f}, {abs(back_lat - center_lat):.8f}")

## 4. Imagery Metadata

In [ ]:
# Create metadata for a hypothetical satellite image
if farm1_aois:
    aoi = farm1_aois[0]
    
    metadata = ImageryMetadata(
        width=2000,
        height=2000,
        bounds=aoi.bounds,
        bands=['coastal_blue', 'blue', 'green', 'red', 'rededge', 'nir'],
        crs="EPSG:4326"
    )
    
    meta_dict = metadata.to_dict()
    
    print("Imagery Metadata:")
    print(f"Dimensions: {meta_dict['width']}x{meta_dict['height']} pixels")
    print(f"Bands: {', '.join(meta_dict['bands'])}")
    print(f"CRS: {meta_dict['crs']}")
    print(f"Geographic bounds: {meta_dict['bounds']}")
    print(f"Pixel resolution: {meta_dict['pixel_resolution_m'][0]:.2f}m x {meta_dict['pixel_resolution_m'][1]:.2f}m")

## 5. Band Calculations (if NumPy available)

In [ ]:
try:
    import numpy as np
    
    # Create synthetic band data
    height, width = 100, 100
    
    # Simulate bands with realistic value ranges
    red = np.random.rand(height, width) * 0.3  # Red reflectance typically lower
    nir = np.random.rand(height, width) * 0.7  # NIR reflectance typically higher for vegetation
    
    # Calculate NDVI
    ndvi = BandCalculations.calculate_ndvi(red, nir)
    
    print("NDVI Calculation:")
    print(f"Red band: shape={red.shape}, range=[{red.min():.3f}, {red.max():.3f}]")
    print(f"NIR band: shape={nir.shape}, range=[{nir.min():.3f}, {nir.max():.3f}]")
    print(f"NDVI: shape={ndvi.shape}, range=[{ndvi.min():.3f}, {ndvi.max():.3f}]")
    print(f"Mean NDVI: {ndvi.mean():.3f}")
    
    # Normalize NDVI
    ndvi_normalized = BandCalculations.normalize_band(ndvi, method='minmax')
    print(f"\nNormalized NDVI: range=[{ndvi_normalized.min():.3f}, {ndvi_normalized.max():.3f}]")
    
    # Visualize if matplotlib is available
    try:
        import matplotlib.pyplot as plt
        
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
        
        axes[0].imshow(red, cmap='Reds')
        axes[0].set_title('Red Band')
        axes[0].axis('off')
        
        axes[1].imshow(nir, cmap='gray')
        axes[1].set_title('NIR Band')
        axes[1].axis('off')
        
        im = axes[2].imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
        axes[2].set_title('NDVI')
        axes[2].axis('off')
        
        plt.colorbar(im, ax=axes[2], fraction=0.046, pad=0.04)
        plt.tight_layout()
        plt.show()
        
    except ImportError:
        print("\nMatplotlib not available for visualization")
        
except ImportError:
    print("NumPy not available - install with: pip install numpy")

## 6. Full Pipeline Example (if dependencies installed)

In [ ]:
if FULL_LOADER_AVAILABLE:
    from src.data_processing import GeospatialDataPipeline
    
    # Initialize pipeline
    pipeline = GeospatialDataPipeline()
    
    # Load AOIs
    farm1_aois = pipeline.load_aoi_from_geojson(farm1_path)
    
    print("Full Pipeline Loaded Successfully!")
    print(f"Number of AOIs: {len(farm1_aois)}")
    
    for aoi in farm1_aois:
        print(f"\nAOI: {aoi.properties.get('name', 'Unnamed')}")
        print(f"  Accurate area: {aoi.area_m2:.2f} m²")
        print(f"  UTM CRS: {aoi._get_utm_crs()}")
    
    # Example: Prepare synthetic data for training
    import numpy as np
    
    # Create synthetic bands
    height, width = 256, 256
    bands = {
        'red': np.random.rand(height, width) * 0.3,
        'green': np.random.rand(height, width) * 0.4,
        'blue': np.random.rand(height, width) * 0.2,
        'nir': np.random.rand(height, width) * 0.7
    }
    
    # Prepare for training
    training_data = pipeline.prepare_training_data(
        bands,
        normalize=True,
        stack=True,
        band_order=['red', 'green', 'blue', 'nir']
    )
    
    print(f"\nTraining data prepared:")
    print(f"  Shape: {training_data.shape}")
    print(f"  Range: [{training_data.min():.3f}, {training_data.max():.3f}]")
    print(f"  Dtype: {training_data.dtype}")
    
else:
    print("Full loader not available. Install dependencies with:")
    print("pip install -r ../requirements.txt")

## 7. Planet Labs Integration Setup

In [ ]:
import os

# Check if Planet API key is configured
planet_api_key = os.environ.get('PLANET_API_KEY')

if planet_api_key:
    print("✓ Planet API key is configured")
    print(f"  Key length: {len(planet_api_key)} characters")
    
    if FULL_LOADER_AVAILABLE:
        from src.data_processing import SatelliteImageryLoader
        from datetime import datetime, timedelta
        
        # Initialize loader
        imagery_loader = SatelliteImageryLoader(planet_api_key)
        
        # Example search (uncomment to run)
        # if farm1_aois:
        #     results = imagery_loader.search_planet_imagery(
        #         aoi=farm1_aois[0],
        #         start_date=datetime.now() - timedelta(days=30),
        #         end_date=datetime.now(),
        #         cloud_cover_max=0.1
        #     )
        #     print(f"Found {len(results)} imagery items")
        
        print("\nPlanet Labs integration ready!")
    else:
        print("\nInstall dependencies to use Planet integration")
else:
    print("✗ Planet API key not configured")
    print("\nTo use Planet Labs integration:")
    print("1. Get an API key from https://www.planet.com/")
    print("2. Set the environment variable:")
    print("   export PLANET_API_KEY='your-api-key-here'")
    print("3. Restart this notebook")

## Summary

This notebook demonstrated:

1. **Loading GeoJSON files** and extracting AOI information
2. **Converting to Planet Labs format** for API integration
3. **Coordinate conversions** between geographic and pixel coordinates
4. **Managing imagery metadata** for satellite data
5. **Band calculations** including NDVI
6. **Full pipeline usage** (when dependencies are installed)
7. **Planet Labs setup** for satellite imagery access

### Next Steps:

1. Install full dependencies: `pip install -r ../requirements.txt`
2. Add real satellite imagery to test with actual data
3. Configure Planet Labs API key for imagery download
4. Integrate with model training pipeline
5. Develop preprocessing workflows for specific use cases